# Deepwalk

In [ ]:
import random
def deepwalk(log,f1,f2,flag,L):
    #Deepwalk算法，
    print("deepwalk:",f1,f2)
    #构建图
    dic={}
    for item in log[[f1,f2]].values:
        try:
            str(int(item[1]))
            str(int(item[0]))
        except:
            continue
        try:
            dic['item_'+str(int(item[1]))].add('user_'+str(int(item[0])))
        except:
            dic['item_'+str(int(item[1]))]=set(['user_'+str(int(item[0]))])
        try:
            dic['user_'+str(int(item[0]))].add('item_'+str(int(item[1])))
        except:
            dic['user_'+str(int(item[0]))]=set(['item_'+str(int(item[1]))])
    dic_cont={}
    for key in dic:
        dic[key]=list(dic[key])
        dic_cont[key]=len(dic[key])
    print("creating")
    #构建路径
    path_length=10
    sentences=[]
    length=[]
    for key in dic:
        sentence=[key]
        while len(sentence)!=path_length:
            key=dic[sentence[-1]][random.randint(0,dic_cont[sentence[-1]]-1)]
            if len(sentence)>=2 and key == sentence[-2]:
                break
            else:
                sentence.append(key)
        sentences.append(sentence)
        length.append(len(sentence))
        if len(sentences)%100000==0:
            print(len(sentences))
    print(np.mean(length))
    print(len(sentences))
    #训练Deepwalk模型
    print('training...')
    random.shuffle(sentences)
    model = Word2Vec(sentences, size=L, window=5,min_count=5,sg=0, hs=1, workers=10,iter=20)
    print('outputing...')
    #输出
    values=set(log[f1].values)
    w2v=[]
    for v in values:
        try:
            a=[int(v)]
            a.extend(model['user_'+str(int(v))])
            w2v.append(a)
        except:
            pass
    out_df=pd.DataFrame(w2v)
    names=[f1]
    for i in range(L):
        names.append(f1+'_'+ f2+'_'+names[0]+'_deepwalk_embedding_'+str(L)+'_'+str(i))
    out_df.columns = names
    out_df.to_pickle('../emb/' +f1+'_'+ f2+'_deepwalk_'+str(L)+'.pkl') 
    ########################
    values=set(log[f2].values)
    w2v=[]
    for v in values:
        try:
            a=[int(v)]
            a.extend(model['item_'+str(int(v))])
            w2v.append(a)
        except:
            pass
    out_df=pd.DataFrame(w2v)
    names=[f2]
    for i in range(L):
        names.append(f2+'_'+ f1+'_'+names[0]+'_deepwalk_embedding_'+str(L)+'_'+str(i))
    out_df.columns = names
    out_df.to_pickle('../emb/' +f2+'_'+ f1+'_deepwalk_'+str(L)+'.pkl') 

# 向量点积

In [ ]:
deviceid_newsid = ["deviceid_newsid_emb_"+str(i)for i in range(8)]
newsid_deviceid = ["newsid_deviceid_emb_"+str(i)for i in range(8)]
df["cross_vector_sum"] =np.sum(df[deviceid_newsid].values * df[newsid_deviceid].values,axis=1)
deviceid = list(df["deviceid"].unique())
average_metrix = np.zeros((len(deviceid),8))
for index,i in enumerate(deviceid):
    if index%100==0:
        print(index)
    indes = df["deviceid"] == i
    average_metrix[index] = np.mean(df[indes][newsid_deviceid],axis=0)
average_metrix = pd.DataFrame(average_metrix,columns=[i+"_ave"for i in newsid_deviceid])
average_metrix["deviceid"] = deviceid
df = df.merge(average_metrix,on=["deviceid"],how='left')
ave_deviceid = [i+"_ave"for i in newsid_deviceid]
df["ave_deviceid*newsid_deviceid"] =np.sum(df[ave_deviceid].values * df[newsid_deviceid].values,axis=1)

# SHAP筛选特征

In [ ]:
import shap
explainer = shap.TreeExplainer(clf)
# val_x也比较大，这里只抽样其中0.2来测
shap_values = explainer.shap_values(val_x.sample(frac = 0.2))
shap.summary_plot(shap_values, val_x,max_display=30)

global_shap_values = pd.DataFrame(np.abs(shap_values[0]).mean(0),index=val_x.columns).reset_index()
global_shap_values.columns = ['var','feature_importances_']
global_shap_values = global_shap_values.sort_values('feature_importances_',ascending=False)